<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_1_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    747      0 --:--:-- --:--:-- --:--:--   745
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.463704,4.798523,0.229924,11:12


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.663391,4.481355,0.259992,11:07


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.370575,4.444716,0.263931,11:07


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.323857,4.418692,0.266820,11:09


In [0]:
learn.save_encoder('pubmed_fold1_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.733221,3.811896,0.364407,01:16


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.881919,3.385353,0.427361,01:32


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.412111,3.121650,0.464891,03:32


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.222261,2.865472,0.513317,03:31
1,2.017164,2.739475,0.519370,03:30


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.921629,2.635998,0.525424,03:34
1,1.687568,2.541271,0.547216,03:44


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.681367,2.496208,0.561743,03:37
1,1.516149,2.426363,0.564165,03:29


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.454974,2.392923,0.577482,03:41
1,1.405963,2.354990,0.579903,03:46


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.347925,2.316118,0.594431,03:56
1,1.266517,2.291533,0.601695,03:36


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.226170,2.295161,0.598063,03:43
1,1.294527,2.268268,0.599274,03:41
2,1.171769,2.219674,0.612591,03:27
3,1.089737,2.226569,0.611380,03:36


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.110304,2.289148,0.596852,03:34
1,1.145476,2.202574,0.624697,03:33
2,1.087588,2.227226,0.616223,03:28
3,1.012716,2.192895,0.615012,03:46


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.067636,2.287743,0.611380,03:34
1,1.070648,2.211104,0.622276,03:39
2,1.032712,2.237872,0.623487,03:36
3,0.964204,2.233713,0.621065,03:29


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.937724,2.241886,0.621065,03:32
1,0.976194,2.213286,0.618644,03:49
2,0.969457,2.261420,0.622276,03:40
3,0.956733,2.298225,0.628329,03:45


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.941177,2.279042,0.623487,03:40
1,1.010296,2.317097,0.619855,03:35
2,0.939953,2.279181,0.619855,03:30
3,0.880091,2.285249,0.625908,03:36


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.965795,2.363509,0.622276,03:25
1,0.962793,2.343841,0.624697,03:42
2,0.940890,2.379713,0.623487,03:24
3,0.875379,2.375241,0.627119,03:37


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.960980,2.399714,0.618644,03:34
1,0.970428,2.387114,0.611380,03:33
2,0.932894,2.345203,0.622276,03:25
3,0.887508,2.358539,0.622276,03:35


In [0]:
# save the best model

learn.save_encoder('ask-1-tuning-pubmed')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[37, 129, 8, 817, 8, 8, 817, 16, 646, 817, 817, 806, 27, 584, 817, 37, 37, 37, 37, 37, 37, 37, 289, 289, 37, 37, 297, 42, 44, 813, 475, 49, 780, 100, 100, 100, 100, 49, 53, 53, 53, 732, 896, 787, 62, 817, 70, 776, 646, 41, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 686, 817, 817, 817, 817, 817, 289, 293, 289, 80, 790, 80, 80, 80, 80, 491, 37, 88, 89, 817, 92, 92, 92, 94, 94, 94, 94, 431, 94, 94, 94, 94, 94, 817, 817, 367, 98, 100, 100, 100, 100, 766, 100, 100, 419, 100, 100, 775, 301, 103, 817, 646, 197, 107, 88, 108, 111, 289, 111, 111, 289, 289, 289, 289, 289, 111, 817, 117, 120, 125, 125, 125, 630, 630, 125, 125, 125, 125, 125, 125, 125, 125, 125, 802, 289, 509, 589, 455, 135, 94, 817, 37, 455, 142, 529, 864, 289, 158, 166, 817, 166, 817, 297, 686, 289, 289, 356, 250, 817, 817, 817, 492, 607, 289, 475, 646, 646, 646, 646, 182, 182, 289, 2, 289, 183, 183, 183, 183, 183, 183, 183, 817, 817, 817, 817, 817, 455

In [44]:
correct_num = 0

for i in range(0, 867):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

449
0.5178777393310265
